# Notebook de procesamiento: Villa de Leyva – Carga robusta, armonización y features

In [1]:
# === Configuración ===
# Ruta del CSV a procesar. Cambia a tu ruta local si lo deseas.
DATA_PATH = "dataset_Recomendacion_villa_de_leyva_eleccion_v2_compania.csv"  # o el archivo que uses
SEP = None               # si conoces el separador, ponlo aquí (por ejemplo ";"). Si no, déjalo en None.
ENC = "utf-8-sig"        # codificación sugerida

print("Usando DATA_PATH =", DATA_PATH)

Usando DATA_PATH = dataset_Recomendacion_villa_de_leyva_eleccion_v2_compania.csv


In [2]:
# === Imports y utilidades ===
import pandas as pd, numpy as np
import unicodedata, re

def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Normaliza encabezados: minúsculas, sin acentos, separadores -> '_'"""
    mapping = {}
    for c in df.columns:
        c2 = unicodedata.normalize("NFKD", str(c)).encode("ascii","ignore").decode("ascii")
        c2 = c2.strip().lower()
        c2 = re.sub(r"[\s\-\./]+", "_", c2)
        c2 = re.sub(r"__+", "_", c2)
        mapping[c] = c2
    return df.rename(columns=mapping)

In [3]:
# === Carga robusta del CSV ===
def read_dataset(path, enc="utf-8-sig", default_sep=None):
    # 1) intenta con el SEP configurado
    if default_sep is not None:
        try:
            df0 = pd.read_csv(path, sep=default_sep, encoding=enc)
            if df0.shape[1] > 1:
                return df0
        except Exception:
            pass
    # 2) autodetección con engine='python'
    try:
        df0 = pd.read_csv(path, sep=None, engine="python", encoding=enc)
        if df0.shape[1] > 1:
            return df0
    except Exception:
        pass
    # 3) fallbacks comunes
    for sep_try in [",",";","\t","|"]:
        try:
            df0 = pd.read_csv(path, sep=sep_try, encoding=enc)
            if df0.shape[1] > 1:
                return df0
        except Exception:
            continue
    # 4) último intento (sin separador)
    return pd.read_csv(path, encoding=enc)

df = read_dataset(DATA_PATH, enc=ENC, default_sep=SEP if 'SEP' in globals() else None)
print("Forma leída:", df.shape)
print("Primeras columnas crudas:", [repr(c) for c in df.columns[:10]])

df = normalize_columns(df)
print("Primeras columnas normalizadas:", [repr(c) for c in df.columns[:10]])

Forma leída: (20984, 25)
Primeras columnas crudas: ["'id_usuario'", "'edad'", "'nacionalidad'", "'origen'", "'tipo_turista_preferido'", "'compañia_viaje'", "'frecuencia_viaje'", "'restricciones_movilidad'", "'presupuesto_estimado'", "'sitios_visitados'"]
Primeras columnas normalizadas: ["'id_usuario'", "'edad'", "'nacionalidad'", "'origen'", "'tipo_turista_preferido'", "'compania_viaje'", "'frecuencia_viaje'", "'restricciones_movilidad'", "'presupuesto_estimado'", "'sitios_visitados'"]


In [4]:
# === Armonización de columnas, sinónimos y tipos ===

# Renombres por sinónimos (ya normalizados)
RENAMES = {
    # fundamentales
    "costo_entrada":      ["costo_entrada","costo","precio_entrada","valor_entrada","entrada","costoentrada","precio","tarifa","tarifa_entrada"],
    "afluencia_promedio": ["afluencia_promedio","afluencia","nivel_afluencia","popularidad"],
    "duracion_esperada":  ["duracion_esperada","duracion","tiempo_recorrido","tiempo_estadia","tiempo_promedio"],
    "admite_mascotas":    ["admite_mascotas","pet_friendly","mascotas","permite_mascotas"],
    # otras posibles
    "compania_viaje":     ["compania_viaje","compañia_viaje"],
    "tipo_sitio":         ["tipo_sitio","tipo","categoria","clasificacion"],
    "ubicacion_geografica":["ubicacion_geografica","ubicacion","municipio","localidad","ciudad"],
    "clima_predominante": ["clima_predominante","clima"],
}

# Función de renombre si está presente
def rename_if_present(target: str, candidates):
    if target in df.columns:
        return
    for cand in candidates:
        if cand in df.columns:
            df.rename(columns={cand: target}, inplace=True)
            return

for tgt, cands in RENAMES.items():
    rename_if_present(tgt, cands)

# Resolver inteligente para costo_entrada si faltara
def _norm_key(s: str) -> str:
    s = unicodedata.normalize("NFKD", str(s or "")).encode("ascii","ignore").decode("ascii")
    s = s.lower().strip()
    s = re.sub(r"[\s\-\./]+","_", s)
    s = re.sub(r"__+","_", s)
    return s

if "costo_entrada" not in df.columns:
    toks_cost = {"costo","precio","valor","tarifa","fee","ticket"}
    toks_ent  = {"entrada","entradas","ticket","boleto","boleta","fee"}
    scored = []
    for col in df.columns:
        parts = set(_norm_key(col).split("_"))
        score = (len(parts & toks_cost) > 0) + (len(parts & toks_ent) > 0)
        if score > 0:
            scored.append((score, col))
    if scored:
        scored.sort(reverse=True)
        _, found = scored[0]
        print(f"ℹ️ Detectado costo en '{found}' -> renombrando a 'costo_entrada'")
        df.rename(columns={found:"costo_entrada"}, inplace=True)
    else:
        print("⚠️ No se detectó costo; se crea costo_entrada=0 (ajústalo si tienes la columna en otra parte)")
        df["costo_entrada"] = 0

# Tipado seguro y rangos plausibles
num_cols_try = {
    "costo_entrada": 0,
    "presupuesto_estimado": None,
    "afluencia_promedio": 3,
    "duracion_esperada": 60,
    "admite_mascotas": 0,
}
for col, fill in num_cols_try.items():
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")
        if fill is not None:
            df[col] = df[col].fillna(fill)
        if col == "afluencia_promedio":
            df[col] = df[col].clip(1,5).round().astype(int)
        if col == "admite_mascotas":
            df[col] = df[col].fillna(0).astype(int)

# Quita posibles fugas (si existe)
if "sitio_recomendado" in df.columns:
    df = df.drop(columns=["sitio_recomendado"])

# Listas base (sin acentos en nombres de columnas)
CAT_COLS = [
    "nacionalidad","origen","tipo_turista_preferido","compania_viaje",
    "restricciones_movilidad","nombre_sitio","tipo_sitio","accesibilidad_general",
    "idioma_info","ubicacion_geografica","clima_predominante","epoca_visita"
]
NUM_COLS = [
    "edad","frecuencia_viaje","presupuesto_estimado","sitios_visitados",
    "calificacion_sitios_previos","tiempo_estancia_promedio","costo_entrada",
    "afluencia_promedio","duracion_esperada","admite_mascotas"
]

missing = [c for c in (CAT_COLS+NUM_COLS) if c not in df.columns]
if missing:
    print("⚠️ Faltan columnas tras armonización:", missing)
print("Columnas disponibles (normalizadas):", df.columns.tolist())

Columnas disponibles (normalizadas): ['id_usuario', 'edad', 'nacionalidad', 'origen', 'tipo_turista_preferido', 'compania_viaje', 'frecuencia_viaje', 'restricciones_movilidad', 'presupuesto_estimado', 'sitios_visitados', 'calificacion_sitios_previos', 'tiempo_estancia_promedio', 'nombre_sitio', 'tipo_sitio', 'costo_entrada', 'afluencia_promedio', 'duracion_esperada', 'accesibilidad_general', 'admite_mascotas', 'idioma_info', 'ubicacion_geografica', 'clima_predominante', 'epoca_visita', 'rating_usuario']


In [5]:
# === Features robustas (AFINIDAD + make_features) ===
def _norm_txt(s):
    s = str(s or "").lower()
    s = unicodedata.normalize("NFKD", s).encode("ascii","ignore").decode("ascii")
    return s

AFINIDAD = {
    "cultural":   {"museo":0.9,"historico":0.9,"religioso":0.7,"arquitectura":0.85,"museo_religioso":0.8,"arqueologico":0.85,"plaza":0.7},
    "naturaleza": {"natural":0.95,"senderismo":0.9,"mirador":0.8,"parque_urbano":0.6},
    "aventura":   {"aventura":0.95,"senderismo":0.85,"parque_tematico":0.7},
    "gastronomia":{"gastronomico":0.95,"enoturismo":0.9,"artesanal":0.6,"plaza":0.6},
    "relax":      {"mirador":0.9,"plaza":0.8,"arquitectura":0.8,"natural":0.75},
    "fotografia": {"mirador":0.9,"plaza":0.8,"arquitectura":0.8,"natural":0.75},
}
_alias_tipo_tur = {"gastronomico":"gastronomia","gastronomia":"gastronomia","relax_fotografia":"relax"}

def make_features(X: pd.DataFrame) -> pd.DataFrame:
    X = X.copy()
    # ratio costo/presupuesto (a prueba de faltantes)
    presu = X.get("presupuesto_estimado", 1).astype(float).replace(0, 1.0)
    costo = X.get("costo_entrada", 0).astype(float).fillna(0.0)
    X["ratio_costo_presu"] = (costo / (presu * 0.15)).clip(0, 3)

    # afinidad perfil × tipo_sitio
    def _afin(r):
        tt = _norm_txt(r.get("tipo_turista_preferido",""))
        ts = _norm_txt(r.get("tipo_sitio",""))
        tt = _alias_tipo_tur.get(tt, tt)
        return AFINIDAD.get(tt, {}).get(ts, 0.5)
    X["afinidad_tipo"] = X.apply(_afin, axis=1)

    # cruces categóricos
    X["x_tipoTur__tipoSit"] = X["tipo_turista_preferido"].astype(str) + "×" + X["tipo_sitio"].astype(str)
    X["x_epoca__tipoSit"]   = X["epoca_visita"].astype(str) + "×" + X["tipo_sitio"].astype(str)
    return X

In [6]:
# === Aplicar features, crear objetivo y listas extendidas ===
df = make_features(df)

# Objetivo binario para clasificación (si está rating_usuario)
if "rating_usuario" in df.columns:
    df["y_like"] = (df["rating_usuario"] >= 4.0).astype(int)
else:
    df["y_like"] = np.nan  # si no hay rating, queda vacío y puedes crear otro objetivo

CAT_COLS_X = CAT_COLS + ["x_tipoTur__tipoSit","x_epoca__tipoSit"]
NUM_COLS_X = NUM_COLS + ["ratio_costo_presu","afinidad_tipo"]

print("Filas:", len(df))
print("Ejemplo columnas categóricas (X):", CAT_COLS_X[:6], "... total", len(CAT_COLS_X))
print("Ejemplo columnas numéricas (X):", NUM_COLS_X[:6], "... total", len(NUM_COLS_X))
df.head(3)

Filas: 20984
Ejemplo columnas categóricas (X): ['nacionalidad', 'origen', 'tipo_turista_preferido', 'compania_viaje', 'restricciones_movilidad', 'nombre_sitio'] ... total 14
Ejemplo columnas numéricas (X): ['edad', 'frecuencia_viaje', 'presupuesto_estimado', 'sitios_visitados', 'calificacion_sitios_previos', 'tiempo_estancia_promedio'] ... total 12


,id_usuario,edad,nacionalidad,origen,tipo_turista_preferido,compania_viaje,frecuencia_viaje,restricciones_movilidad,presupuesto_estimado,sitios_visitados,...,idioma_info,ubicacion_geografica,clima_predominante,epoca_visita,rating_usuario,ratio_costo_presu,afinidad_tipo,x_tipoTur__tipoSit,x_epoca__tipoSit,y_like
0,U96685,27,Colombia,Pasto,historia,solo,3,ninguna,390920,6,...,es,Santa Sofía,templado_seco,puente_festivo,3.9,0.193833,0.5,historia×museo,puente_festivo×museo,0
1,U96685,27,Colombia,Pasto,historia,solo,3,ninguna,390920,6,...,es,Ráquira,seco,vacaciones_fin_de_año,4.9,0.119377,0.5,historia×gastronomico,vacaciones_fin_de_año×gastronomico,1
2,U96685,27,Colombia,Pasto,historia,solo,3,ninguna,390920,6,...,es,Santa Sofía,templado_seco,puente_festivo,4.0,0.075241,0.5,historia×museo,puente_festivo×museo,1


In [7]:
# === Validaciones rápidas y guardado opcional ===
def check_nulls(d, cols):
    s = d[cols].isna().sum().sort_values(ascending=False)
    return s[s>0]

nulls_cat = check_nulls(df, [c for c in CAT_COLS_X if c in df.columns])
nulls_num = check_nulls(df, [c for c in NUM_COLS_X if c in df.columns])

print("Nulos en categóricas (top):\n", nulls_cat.head(10))
print("Nulos en numéricas (top):\n", nulls_num.head(10))

# Guardar versión limpia/featurizada
OUT_PATH = "dataset_vdl_ready_features.csv"
df.to_csv(OUT_PATH, index=False, encoding="utf-8-sig")
print("Guardado:", OUT_PATH)

Nulos en categóricas (top):
 Series([], dtype: int64)
Nulos en numéricas (top):
 Series([], dtype: int64)
Guardado: dataset_vdl_ready_features.csv
